In [1]:
import os
os.chdir("/dust3r")
print(os.getcwd())

/dust3r


In [18]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
import json
import open3d as o3d

from dust3r.inference import inference_with_mask
from dust3r.model import AsymmetricCroCo3DStereo
from dust3r.utils.image import load_images
from dust3r.image_pairs import make_pairs
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode
from dust3r.cloud_opt.base_opt import global_alignment_loop
from masked_dust3r.scripts.utils.math import *
from masked_dust3r.scripts.utils.image import *


DATA_PATH = "/dust3r/masked_dust3r/data/jackal_dense"
IMG_FILE_EXTENSION = ".png"
MASK_FILE_EXTENSION = ".png"
GAUSSIAN_SIGMA = 51.0 #TODO: Dynamically size GAUSSIAN_SIGMA based on mask size?
INIT_FRAMES = 50
NEW_FRAMES = 10
PREVIOUS_FRAMES = 40
TOTAL_FRAMES = 300

INIT_WEIGHT_FOCAL = 0.1
INIT_WEIGHT_Z = 0.0001 
INIT_WEIGHT_ROT = 0.0001 * 0
INIT_WEIGHT_TRANS_SMOOTHNESS = 0.0001 * 0
INIT_WEIGHT_ROT_SMOOTHNESS = 0.001 * 0

NEW_WEIGHT_FOCAL = 0.1
NEW_WEIGHT_Z = 0.1
NEW_WEIGHT_ROT = 0.1
NEW_WEIGHT_TRANS_SMOOTHNESS = 0.00001
NEW_WEIGHT_ROT_SMOOTHNESS = 0.00001

USE_COMMON_INTRINSICS = False
IS_FOCAL_FIXED = True
FOCAL_LENGTH = 4.74

device = 'cuda'
batch_size = 1
schedule = 'cosine'
lr = 0.01
niter = 300

In [3]:
# Load the model

model_name = "checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth"
# you can put the path to a local checkpoint in model_name if needed
model = AsymmetricCroCo3DStereo.from_pretrained(model_name).to(device)

... loading model from checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth


instantiating : AsymmetricCroCo3DStereo(enc_depth=24, dec_depth=12, enc_embed_dim=1024, dec_embed_dim=768, enc_num_heads=16, dec_num_heads=12, pos_embed='RoPE100', patch_embed_cls='PatchEmbedDust3R', img_size=(512, 512), head_type='dpt', output_mode='pts3d', depth_mode=('exp', -inf, inf), conf_mode=('exp', 1, inf), landscape_only=False)
<All keys matched successfully>


In [4]:
images_array = []
masks_array = []

for i in range(0,20):
    images_array.append(os.path.join(DATA_PATH,"masked_images/{}{}".format(i,IMG_FILE_EXTENSION)))
    masks_array.append(os.path.join(DATA_PATH,"masks/{}{}".format(i,MASK_FILE_EXTENSION)))
images = load_images(images_array, size=512, verbose=True)
_,_,H,W = images[0]["img"].shape
masks = load_masks(masks_array, H, W, device)

>> Loading a list of 20 images
 - adding /dust3r/masked_dust3r/data/jackal_dense/masked_images/0.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_dense/masked_images/1.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_dense/masked_images/2.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_dense/masked_images/3.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_dense/masked_images/4.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_dense/masked_images/5.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_dense/masked_images/6.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_dense/masked_images/7.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_dense/masked_images/8.png with resolution 1280x720 --> 512x288
 - add

In [5]:
pairs = make_pairs(images, scene_graph='swin-5', prefilter=None, symmetrize=True)
output = inference_with_mask(pairs, model, device, masks, GAUSSIAN_SIGMA, batch_size=batch_size)
del pairs

>> Inference with model on 200 image pairs


  0%|          | 0/200 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 200/200 [01:03<00:00,  3.13it/s]


In [6]:
#init_scene = global_aligner(output, device=device, mode=GlobalAlignerMode.PlanePointCloudOptimizer)
#loss = init_scene.compute_global_alignment(init="mst", niter=niter, schedule='cosine', lr=lr)

#scene = init_scene

scene = global_aligner(output, device=device, mode=GlobalAlignerMode.PlanePointCloudOptimizer, 
                        weight_focal = INIT_WEIGHT_FOCAL,
                        weight_z = INIT_WEIGHT_Z ,
                        weight_rot = INIT_WEIGHT_ROT  ,
                        weight_trans_smoothness = INIT_WEIGHT_TRANS_SMOOTHNESS,
                        weight_rot_smoothness = INIT_WEIGHT_ROT_SMOOTHNESS)
#scene.im_poses = calculate_new_params(init_scene.im_poses,device)
#scene.im_focals = init_scene.im_focals
loss = scene.compute_global_alignment(init="mst", niter=niter, schedule=schedule, lr=lr)

#averge_focal = scene.get_focals().sum().item()/len(images_array)
#fixed_focal = [averge_focal for _ in range(len(images_array))]
#mask = [True for _ in range(len(images_array))]
#scene.preset_focal(fixed_focal, mask)
#loss = scene.compute_global_alignment(init="mst", niter=niter, schedule=schedule, lr=lr)

#scene.weight_focal = INIT_WEIGHT_FOCAL
#scene.weight_z = INIT_WEIGHT_Z 
#scene.weight_rot = INIT_WEIGHT_ROT
#scene.weight_trans_smoothness = INIT_WEIGHT_TRANS_SMOOTHNESS 
#scene.weight_rot_smoothness = INIT_WEIGHT_ROT_SMOOTHNESS
#loss = global_alignment_loop(scene, niter=niter, schedule=schedule, lr=lr)


 init edge (7*,12*) score=1.204464316368103
 init edge (8*,12) score=1.1818453073501587
 init edge (7,11*) score=1.176344394683838
 init edge (7,5*) score=1.1713817119598389
 init edge (5,10*) score=1.170652151107788
 init edge (16*,12) score=1.1688141822814941
 init edge (12,17*) score=1.1647253036499023
 init edge (7,4*) score=1.1444718837738037
 init edge (2*,17) score=1.1384222507476807
 init edge (16,1*) score=1.1216689348220825
 init edge (8,13*) score=1.1856962442398071
 init edge (6*,11) score=1.1774686574935913
 init edge (14*,10) score=1.1745556592941284
 init edge (10,15*) score=1.1719152927398682
 init edge (9*,14) score=1.163331151008606
 init edge (4,19*) score=1.1493381261825562
 init edge (3*,19) score=1.1466799974441528
 init edge (15,0*) score=1.1243374347686768
 init edge (3,18*) score=1.1607537269592285
 init loss = 0.5039066076278687
Global alignement - optimizing for:
['pw_poses', 'im_depthmaps.0', 'im_depthmaps.1', 'im_depthmaps.2', 'im_depthmaps.3', 'im_depthmap

100%|██████████| 300/300 [02:05<00:00,  2.38it/s, lr=1.27413e-06 loss=0.318966]


In [15]:
imgs = scene.imgs
focals = scene.get_focals()
poses = scene.get_im_poses()
pts3d = scene.get_pts3d()
confidence_masks = scene.get_masks()
intrinsics = scene.get_intrinsics()

In [19]:
#Check if pointclouds folder exists
#If exists, delete all files in the folder
if os.path.exists("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH)):
    for file in os.listdir("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH)):
        os.remove("{DATA_PATH}/pointclouds/{file}".format(DATA_PATH=DATA_PATH, file=file))
        
if not os.path.exists("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH)):
    os.makedirs("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH))

for i in range(len(images)):
    pointcloud = pts3d[i].detach().cpu().numpy()
    pointcloud = pointcloud.reshape(-1, 3)
    color = imgs[i].reshape(-1, 3)
    confidence_mask = confidence_masks[i].detach().cpu().numpy()
    confidence_mask = confidence_mask.reshape(-1)
    
    masked_pointcloud = []
    masked_color = []

    for j in range(len(confidence_mask)):
        if confidence_mask[j]:
            masked_pointcloud.append(pointcloud[j])
            masked_color.append(color[j])

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(masked_pointcloud)
    pcd.colors = o3d.utility.Vector3dVector(masked_color)
    o3d.io.write_point_cloud("{DATA_PATH}/pointclouds/pointcloud{i}.ply".format(DATA_PATH=DATA_PATH, i=i), pcd)

In [20]:
#Create transform file
#TODO: Per frame camera model?
transforms = {}
transforms["camera_model"] = "OPENCV"
if USE_COMMON_INTRINSICS:
    averge_focal = focals.sum()/len(focals)
    transforms["fl_x"] = averge_focal.item()
    transforms["fl_y"] = averge_focal.item()

    #Find size of images
    img = Image.open(images_array[0])
    width, height = img.size
    transforms["w"] = width
    transforms["h"] = height
    transforms["cx"] = width//2
    transforms["cy"] = height//2

transforms["frames"] = []

OPENGL = np.array([[1, 0, 0, 0],
                    [0, -1, 0, 0],
                    [0, 0, -1, 0],
                    [0, 0, 0, 1]])

for i in range(len(poses)):
    if not((confidence_masks[i]==0).all()):
        frame = {}
        frame["file_path"] = "/".join(images_array[i].split("/")[-2:])
        #frame["transform_matrix"] = np.linalg.inv(poses[i].detach().cpu().numpy()).tolist()
        frame["transform_matrix"] = np.dot(poses[i].detach().cpu().numpy(),OPENGL).tolist()
        frame["mask_path"] = "/".join(masks_array[i].split("/")[-2:])
        transforms["frames"].append(frame)
        
        if not USE_COMMON_INTRINSICS:
            frame["fl_x"] = intrinsics[i,0,0].item()
            frame["fl_y"] = intrinsics[i,1,1].item()
            frame["cx"] = intrinsics[i,0,2].item()
            frame["cy"] = intrinsics[i,1,2].item()
            img = Image.open(images_array[i])
            width, height = img.size
            transforms["w"] = width
            transforms["h"] = height

#Save transform file
with open("{}/transforms.json".format(DATA_PATH), 'w') as f:
    json.dump(transforms, f, indent=4)

In [11]:
print(transforms["frames"])

[{'file_path': 'masked_images/0.png', 'transform_matrix': [[0.9452298283576965, 0.25495293736457825, -0.20381249487400055, -0.058613043278455734], [0.25287798047065735, -0.9668051600456238, -0.03661201149225235, -0.009042802266776562], [-0.20638130605220795, -0.016932928934693336, -0.9783251285552979, 0.03645485267043114], [0.0, 0.0, 0.0, 1.0]], 'mask_path': 'masks/0.png'}, {'file_path': 'masked_images/1.png', 'transform_matrix': [[0.9639269113540649, 0.19710680842399597, -0.17886775732040405, -0.05059170722961426], [0.19258451461791992, -0.9803603887557983, -0.042480118572711945, -0.010273641906678677], [-0.18372800946235657, 0.006500573828816414, -0.9829555749893188, 0.03507082909345627], [0.0, 0.0, 0.0, 1.0]], 'mask_path': 'masks/1.png'}, {'file_path': 'masked_images/2.png', 'transform_matrix': [[0.9765200614929199, 0.17060469090938568, -0.13153930008411407, -0.03962356597185135], [0.17624182999134064, -0.9838138818740845, 0.03238897770643234, 0.008964123204350471], [-0.123884484171

In [12]:
import roma

all_poses = torch.stack(list(scene.im_poses))
Q = all_poses[:,:4]
Q = torch.nn.functional.normalize(Q, p=2, dim=1)
T = signed_expm1(all_poses[:,4:7])
tf = roma.RigidUnitQuat(Q, T).normalize().to_homogeneous()

OPENGL = torch.tensor([[1, 0, 0, 0],
                       [0, -1, 0, 0],
                       [0, 0, -1, 0],
                       [0, 0, 0, 1]], dtype=torch.float32).to(device)

tf = torch.matmul(tf, OPENGL)

tf = roma.RigidUnitQuat(Q, T).normalize()
print(tf.linear[:,0]/tf.linear[:,3])

tensor([-0.0051, -0.0125,  0.0221,  0.0162,  0.0238,  0.0079, -0.0086, -0.0022,
        -0.0034, -0.0124, -0.0041, -0.0207,  0.0064, -0.0098, -0.0237,  0.0138,
        -0.0075, -0.0030, -0.0227, -0.0210], device='cuda:0',
       grad_fn=<DivBackward0>)
